# **Task 3. Neural classification **
Comparison of  soft- hard- and neural algorithms
As a target metric for estimation I've chosen AUC-ROC score, because it is basically one of the classic measurements saying how good our model is when distinguishing classes. So, the model with a higher meaning will be considered to be the best one.

The models we are going to constrauct are the following:
1. Fasttext model with the score of 91,7%
2. "Hard baseline" of TFIDF + XGBoost. Inn this particular version it didn't finish calculations due to the end of collab time. But the only time it performed it gave me around 88%, which is a lower result. Taking into consideration time it takes.... well I have no idea how to use it with really huge samples
3. Real NN - still in process... and I have no idea how much time it can take

Small spoiler: really big models do not run because of the limited computing power i have (even usinng Google collab - 8h not enough). But still doestn't mean that they wouldn't work!)

In [0]:
! pip install fasttext

     |████████████████████████████████| 71kB 2.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3016225 sha256=16b8f479670f261916dc0b03c5898def5faa50fa52023833b15ff0d0a3596564
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


# **3.1 Fasttext classifier**

In [0]:
import numpy as np 
import pandas as pd 
import fasttext
import bz2
import csv
from sklearn.metrics import roc_auc_score
import os

In [0]:
train=bz2.BZ2File("train.ft.txt.bz2")
train=train.readlines()
train=[x.decode('utf-8') for x in train]
print(len(train))

3600000


In [0]:
test=bz2.BZ2File("test.ft.txt.bz2")
test=test.readlines()
test=[x.decode('utf-8') for x in test]
print(len(test))

400000


In [0]:
#we prepare our training sample
train = pd.DataFrame(train)
train.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

# after, we build a model itself
model1 = fasttext.train_supervised('train.txt',label_prefix='__label__', thread=4, epoch = 10)
print(model1.labels, 'the labels')

['__label__1', '__label__2'] the labels


In [0]:
#now we prepare the test data
# firstly, we have to get rid of labels from the sample  
testnew = [w.replace('__label__2 ', '') for w in test]
testnew = [w.replace('__label__1 ', '') for w in testnew]
testnew = [w.replace('\n', '') for w in testnew]

# Use the predict function 
pred = model1.predict(testnew)

In [0]:
# he we channge the labels from test sample and the model predictions ones and zeros
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

# we use the roc auc curve as a measurement because it demonstrates how good our model is 
print(roc_auc_score(labels, pred_labels))

0.9173650000000001


We got a result of 91,7% 
Will compare this to hardbaseline model soon

# **3.2 TFIDF + XGBoost classifier**

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
! pip install fasttext

     |████████████████████████████████| 71kB 2.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3016215 sha256=70bb3d25b824c112f7c08d990a94fbddf9f25a82274e0be1230827296bc5df24
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [0]:

import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import fasttext
import string
import re
import bz2
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.preprocessing.text import Tokenizer

from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk

Using TensorFlow backend.


In [0]:
train=bz2.BZ2File("train.ft.txt.bz2")
train=train.readlines()
train=[x.decode('utf-8') for x in train]
print(len(train))

3600000


In [0]:
test=bz2.BZ2File("test.ft.txt.bz2")
test=test.readlines()
test=[x.decode('utf-8') for x in test]
print(len(test))

400000


In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
def Tokenizerr(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [0]:
vectorizer = TfidfVectorizer(tokenizer=Tokenizerr, stop_words=stop_words)

In [0]:
import csv

In [0]:
#we prepare our training sample
train = pd.DataFrame(train)
train.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [0]:
f = open('train.txt', 'r', encoding ='utf-8')
X_train = []
for i in f: 
    X_train.append(i[11:])
f.close()

In [0]:
f = open('train.txt', 'r', encoding ='utf-8')
y_train = []
for i in f: 
    if i[:10] == '__label__1':
        y_train.append(0)
    else:
        y_train.append(1)
f.close()

In [0]:
test = pd.DataFrame(test)
test.to_csv("test.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [0]:
f = open('test.txt', 'r', encoding ='utf-8')
X_test = []
for i in f: 
    X_test.append(i[11:])
f.close()

In [0]:
f = open('test.txt', 'r', encoding ='utf-8')
y_test = []
for i in f: 
    if i[:10] == '__label__1':
        y_test.append(0)
    else:
        y_test.append(1)
f.close()

In [0]:
train = vectorizer.fit_transform(X_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Well, we have a warning here that some of the tokens generated are not apparently reflect the appropriate lannguage

In [0]:
test = vectorizer.transform(X_test)

In [0]:
XGB_model = XGBClassifier()

In [0]:
XGB_model.fit(train, y_train)

XGB_model.save_model("XGB_model.bin")

In [0]:
XGB_model.load_model("XGB_model.bin")

In [0]:
XGB_pred = XGB_model.predict(test)

with open('XGB_pred', 'wb') as f:
     pickle.dump(XGB_pred, f)

In [0]:
with open('XGB_pred', 'rb') as f:
     XGB_pred = pickle.load(f)

In [0]:
XGB_auc = roc_auc_score(y_test, XGB_pred)

In [0]:
print ("SCORE:", XGB_auc)

Here is the performance of our XGBoost...

Actually, I've spent days running and running this stuff again. And every single time the process was interrupted. It really sucks. The one time I got a result - it was a bit lower than 88%.

## **3.3 NN**

In [0]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
import bz2

In [0]:
import csv

In [0]:
train=bz2.BZ2File("train.ft.txt.bz2")
train=train.readlines()
train=[x.decode('utf-8') for x in train]

In [0]:
test=bz2.BZ2File("test.ft.txt.bz2")
test=test.readlines()
test=[x.decode('utf-8') for x in test]

In [0]:
#we prepare our training sample
train = pd.DataFrame(train)
train.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

As previously, we transform it to fit in the model than 

In [0]:
f = open('train.txt', 'r', encoding ='utf-8')
X_train = []
for i in f: 
    X_train.append(i[11:])
f.close()

In [0]:
f = open('train.txt', 'r', encoding ='utf-8')
y_train = []
for i in f: 
    if i[:10] == '__label__1':
        y_train.append(0)
    else:
        y_train.append(1)
f.close()

In [0]:
test = pd.DataFrame(test)
test.to_csv("test.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [0]:
f = open('test.txt', 'r', encoding ='utf-8')
X_test = []
for i in f: 
    X_test.append(i[11:])
f.close()

In [0]:
f = open('test.txt', 'r', encoding ='utf-8')
y_test = []
for i in f: 
    if i[:10] == '__label__1':
        y_test.append(0)
    else:
        y_test.append(1)
f.close()

In [0]:
train = vectorizer.fit_transform(X_train)

In [0]:
test = vectorizer.transform(X_test)

In [0]:
#padding
max_words = 500
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(train)
sequences = tok.texts_to_sequences(train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
#we define a function for building a recurrent NN 
def recNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Dense(256,name='FC1')(layer)
    layer = LSTM(64)(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Activation('sigmoid')(layer)
    model3 = Model(inputs=inputs,outputs=layer)
    return model3

In [0]:
model3 = recNN()
model3.summary()
model3.compile(loss='binary_crossentropy',optimizer="sgd",metrics=['accuracy'])

In [0]:
# fit the data to RNN with batch size of 128 and 5 epochs
model.fit(sequences_matrix,y_train,batch_size=128,epochs=5, validation_split=0.2)

Here, in an ideal world, we get a AUC ROC score and try to compare it with other results. But realistically, I've spend several nights trying to figure out how to make it work. No chance.
Any advice on how to do it? Theoretically, it will work on more powerfull machines.